기타 코드

In [ ]:
# 절대 건들지 말 것. 강제 종료 코드
# import os
# os.kill(os.getpid(), 9)

In [ ]:
# 구글 드라이브 데이터 불러오는 코드
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


- kresnik/wav2vec2-large-xlsr-korean를 적용한 코드

- 해당 모델은 음성 인식 파트인데 임베딩하는 것까지 만 사용할 것

- 다른 코드랑 병행하면 코드가 복잡해지므로 json으로 추출해서 사용할 것

In [ ]:
import os
import json
import soundfile as sf
import torch
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from transformers.models.wav2vec2.modeling_wav2vec2 import Wav2Vec2PreTrainedModel
import torch.nn as nn

In [ ]:
# 감정 분류 헤드 정의
class RegressionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
# 감정 분류 모델 정의
class EmotionModel(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(self, input_values):
        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)
        return hidden_states, logits

In [ ]:
# 오디오 파일 로드 및 전처리
def load_audio(file_path, expected_sample_rate=16000):
    speech, sr = sf.read(file_path)
    if len(speech.shape) > 1:  # 스테레오 → 모노
        speech = speech.mean(axis=1)
    if sr != expected_sample_rate:
        raise ValueError(f"Sampling rate must be {expected_sample_rate} Hz")
    return speech

In [ ]:
# 임베딩 추출
def extract_embeddings(audio_path, processor, model):
    speech = load_audio(audio_path)
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to(model.device)
    with torch.no_grad():
        outputs = model(input_values)
        hidden_states = outputs.last_hidden_state  # Shape: (Batch, Sequence Length, Hidden Size)
    return hidden_states

In [ ]:
# 감정 예측
def predict_emotion(embeddings, emotion_model):
    # 시퀀스 평균 풀링
    pooled_embeddings = torch.mean(embeddings, dim=1)
    with torch.no_grad():
        _, logits = emotion_model(pooled_embeddings)
    return logits.cpu().numpy()

In [ ]:
# 폴더 내 모든 오디오 파일 탐색 및 처리
def process_audio_files(base_dir, embedding_processor, embedding_model, emotion_model):
    results = {}

    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)
                try:
                    # Step 1: 임베딩 추출
                    embeddings = extract_embeddings(file_path, embedding_processor, embedding_model)

                    # Step 2: 감정 예측
                    emotion_logits = predict_emotion(embeddings, emotion_model)

                    # Step 3: 저장 결과
                    results[file] = {
                        "file_path": file_path,
                        "emotions": {
                            "arousal": float(emotion_logits[0][0]),
                            "dominance": float(emotion_logits[0][1]),
                            "valence": float(emotion_logits[0][2])
                        }
                    }

                except Exception as e:
                    results[file] = {
                        "file_path": file_path,
                        "error": str(e)
                    }
    return results


In [ ]:
# 결과를 JSON으로 저장
def save_results_to_json(results, output_path):
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

In [ ]:
# 메인 함수
def main():
    base_dir = "/content/drive/MyDrive/personal_study/"  # 오디오 파일 폴더 경로
    output_json = "emotion_results.json"  # JSON 파일 저장 경로

    # Step 1: 임베딩 추출을 위한 모델 및 프로세서 로드
    embedding_processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
    embedding_model = Wav2Vec2Model.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to("cuda")

    # Step 2: 감정 분류를 위한 모델 로드
    emotion_processor = Wav2Vec2Processor.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")
    emotion_model = EmotionModel.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim").to("cuda")

    try:
        # Step 3: 폴더 내 오디오 파일 처리
        results = process_audio_files(base_dir, embedding_processor, embedding_model, emotion_model)

        # Step 4: JSON 파일로 저장
        save_results_to_json(results, output_json)
        print(f"Results saved to {output_json}")

    except Exception as e:
        print(f"Error: {e}")

In [ ]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Results saved to emotion_results.json
